In [11]:
//    val input = """RL                    
//
//AAA = (BBB, CCC)
//BBB = (DDD, EEE)
//CCC = (ZZZ, GGG)
//DDD = (DDD, DDD)
//EEE = (EEE, EEE)
//GGG = (GGG, GGG)
//ZZZ = (ZZZ, ZZZ)"""

val input ="""LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""
data class Node(val id: String, val left: String, val right: String) {
    companion object {
        //    AAA = (BBB, CCC)

        fun of(input: String): Node {
            val id = input.substringBefore("=").trim()
            val left = input.substringAfter("(").substringBefore(",").trim()
            val right = input.substringAfter(",").substringBefore(")").trim()
            return Node(id, left, right)
        }
    }
}

data class State(val instructions: List<Char>, val nodes: Map<String, Node>, var currentNodeId: String, var totalSteps: Int = 0, var nextInstructionIdx: Int = 0 ) {
    fun step(): String {
        totalSteps += 1
        val instruction = instructions.elementAt(nextInstructionIdx)
        val currentNode = nodes.get(currentNodeId)
        currentNodeId = if ( instruction == 'L') currentNode!!.left else currentNode!!.right
        nextInstructionIdx = (nextInstructionIdx + 1 ) % instructions.size
        return currentNodeId 
    }

    override fun toString(): String {
        return "$totalSteps: $nextInstructionIdx(${instructions.elementAt(nextInstructionIdx)}),$currentNodeId"
    }
}


var instructions = input.substringBefore("\n").trim().toCharArray().toList()
var nodes = input.substringAfter("\n\n").split("\n").map {Node.of(it)}.groupBy { it.id }.mapValues { (_, list) -> list.single() }

val state = State(instructions, nodes, "AAA")
while (state.currentNodeId != "ZZZ") {
    state.step()
}
println(state.totalSteps)





6
